## 分组
### 一、分组模式及其对象
#### 1. 分组的一般模式

* 想要实现分组操作，需要明确三个要素：
  * 分组依据
  * 数据来源
  * 操作及其返回结果
明确了以上三个方面，就可以确定一个分组操作，分组代码的一般模式为：
```python
df.groupby(分组依据)[数据来源].使用操作
```



In [4]:
# 在colab中进行到数据目录进行操作
!ls
%cd drive/MyDrive
!ls

drive  sample_data
/content/drive/MyDrive
'Colab Notebooks'   data


In [5]:
import numpy as np
import pandas as pd

In [7]:
#按照性别统计身高中位数
df = pd.read_csv('data/learn_pandas.csv')
df.groupby('Gender')['Height'].median()

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

In [8]:
df.head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
3,Fudan University,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N,2,2020/1/3,0:04:08
4,Fudan University,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22


#### 2. 分组依据的本质
如果是根据多个维度进行分组，只需在`groupby`中传入相应列名构成的列表即可。

In [9]:
#根据学校和性别进行分组，统计身高的均值
df.groupby(['School','Gender'])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

`groupby`的分组依据都是直接可以从列中按照名字获取的，如果是通过一定的复杂逻辑来分组，就应该先写出分组条件，然后再将其传入`groupby`中：

In [16]:
#分组条件
condition = df.Weight > df.Weight.mean()

In [17]:
# 然后将其传入goupby
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

In [12]:
# 按照上下四分位数分割，将体重分为high、normal、low三组，统计身高的均值
# 第一四分位数
low_condition = df.Weight > df.Weight.quantile(.25)
df.groupby(low_condition)['Height'].mean()




Weight
False    155.891071
True     166.448819
Name: Height, dtype: float64

In [18]:
# 第二四分位数
# 等价于中位数
normal_condition = df.Weight > df.Weight.quantile(.5)
df.groupby(normal_condition)['Height'].mean()


Weight
False    157.811881
True     169.876829
Name: Height, dtype: float64

In [19]:
# 第三四分位数
high_condition = df.Weight > df.Weight.quantile(.75)
df.groupby(high_condition)['Height'].mean()


Weight
False    159.727660
True     174.935714
Name: Height, dtype: float64

> 从索引来看，最后产生的结果就是按照条件列表中元素的值（也就是`True`和`False`）来分组。

In [21]:
item = np.random.choice(list('abc'), df.shape[0])
print(item)
df.groupby(item)['Height'].mean()

['c' 'b' 'a' 'c' 'b' 'a' 'c' 'c' 'c' 'a' 'c' 'a' 'a' 'a' 'c' 'c' 'c' 'a'
 'a' 'a' 'c' 'a' 'c' 'b' 'a' 'a' 'c' 'b' 'a' 'a' 'b' 'a' 'c' 'b' 'b' 'b'
 'a' 'b' 'b' 'b' 'a' 'a' 'c' 'a' 'b' 'b' 'b' 'a' 'a' 'a' 'b' 'a' 'b' 'a'
 'a' 'a' 'b' 'b' 'b' 'a' 'b' 'b' 'c' 'c' 'a' 'b' 'c' 'a' 'a' 'a' 'b' 'b'
 'c' 'c' 'a' 'b' 'b' 'a' 'a' 'b' 'c' 'b' 'c' 'c' 'b' 'b' 'a' 'b' 'b' 'c'
 'a' 'a' 'c' 'b' 'b' 'b' 'b' 'c' 'a' 'c' 'b' 'c' 'c' 'a' 'b' 'b' 'b' 'a'
 'a' 'b' 'b' 'a' 'a' 'c' 'a' 'c' 'a' 'a' 'b' 'a' 'b' 'b' 'c' 'c' 'c' 'b'
 'b' 'b' 'b' 'b' 'a' 'a' 'a' 'c' 'c' 'a' 'a' 'c' 'a' 'c' 'a' 'a' 'b' 'c'
 'c' 'a' 'b' 'b' 'b' 'c' 'a' 'b' 'c' 'b' 'b' 'b' 'c' 'b' 'c' 'b' 'b' 'a'
 'c' 'b' 'b' 'b' 'c' 'a' 'b' 'b' 'c' 'a' 'a' 'a' 'b' 'b' 'b' 'c' 'b' 'a'
 'a' 'c' 'c' 'c' 'a' 'b' 'a' 'c' 'a' 'a' 'b' 'c' 'a' 'a' 'a' 'b' 'c' 'c'
 'a' 'b']


a    164.214062
b    162.728169
c    162.614583
Name: Height, dtype: float64

如果传入多个序列进入`groupby`，那么最后分组的依据就是这两个序列对应行的唯一组合：


In [23]:
df.groupby([condition, item])['Height'].mean()

Weight   
False   a    159.247500
        b    159.288679
        c    158.388235
True    a    172.491667
        b    172.855556
        c    172.878571
Name: Height, dtype: float64

从上例中可以看出，之前传入列名只是一种简便的记号，实际上等价于传入的是一个或多个列，最后分组依据来自于数据来源组合的unique值，通过`drop_duplicates`就能知道具体的组类别。

In [24]:
#drop_duplicates()用于去重
df[['School', 'Gender']].drop_duplicates()

,School,Gender
0,Shanghai Jiao Tong University,Female
1,Peking University,Male
2,Shanghai Jiao Tong University,Male
3,Fudan University,Female
4,Fudan University,Male
5,Tsinghua University,Female
9,Peking University,Female
16,Tsinghua University,Male


In [25]:
df.groupby([df['School'], df['Gender']])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

#### 3. Groupby对象
最终具体做分组操作时，所调用的方法都来自于`pandas`中的`groupby`对象。因此也就具有一些方便的属性。

In [26]:
gb = df.groupby(['School','Grade'])

gb

In [27]:
# ngroups属性:分组个数
gb.ngroups

16

In [30]:
#groups属性，可以返回从组名映射到组索引列表的字典
res=gb.groups
# res.keys()
print(res)

{('Fudan University', 'Freshman'): [15, 28, 63, 70, 73, 105, 108, 157, 186], ('Fudan University', 'Junior'): [26, 41, 82, 84, 90, 107, 145, 152, 173, 187, 189, 195], ('Fudan University', 'Senior'): [39, 46, 49, 52, 66, 77, 112, 129, 131, 138, 144], ('Fudan University', 'Sophomore'): [3, 4, 37, 48, 68, 98, 135, 170], ('Peking University', 'Freshman'): [1, 32, 35, 36, 38, 45, 54, 57, 88, 96, 99, 140, 185], ('Peking University', 'Junior'): [9, 20, 59, 72, 75, 102, 159, 183], ('Peking University', 'Senior'): [30, 86, 116, 127, 130, 132, 147, 194], ('Peking University', 'Sophomore'): [29, 61, 83, 101, 120], ('Shanghai Jiao Tong University', 'Freshman'): [0, 6, 10, 60, 114, 117, 119, 121, 141, 148, 149, 153, 184], ('Shanghai Jiao Tong University', 'Junior'): [31, 42, 50, 56, 58, 64, 85, 93, 115, 122, 143, 155, 164, 172, 174, 188, 190], ('Shanghai Jiao Tong University', 'Senior'): [2, 12, 19, 21, 22, 23, 79, 87, 89, 103, 104, 109, 123, 134, 156, 161, 165, 166, 171, 192, 197, 198], ('Shanghai 

In [33]:
# 上一小节介绍了可以通过 drop_duplicates 得到具体的组类别，
# 现请用 groups 属性完成类似的功能。
print('School, Grade')
for key in res.keys():
  print('{}, {}'.format(key[0], key[1]))


School, Grade
Fudan University, Freshman
Fudan University, Junior
Fudan University, Senior
Fudan University, Sophomore
Peking University, Freshman
Peking University, Junior
Peking University, Senior
Peking University, Sophomore
Shanghai Jiao Tong University, Freshman
Shanghai Jiao Tong University, Junior
Shanghai Jiao Tong University, Senior
Shanghai Jiao Tong University, Sophomore
Tsinghua University, Freshman
Tsinghua University, Junior
Tsinghua University, Senior
Tsinghua University, Sophomore


当`size`作为`DataFrame`的属性时，返回的是表长乘以表宽的大小，但在`groupby`对象上表示的是统计每个组的元素个数：

In [34]:
gb.size()

School                         Grade    
Fudan University               Freshman      9
                               Junior       12
                               Senior       11
                               Sophomore     8
Peking University              Freshman     13
                               Junior        8
                               Senior        8
                               Sophomore     5
Shanghai Jiao Tong University  Freshman     13
                               Junior       17
                               Senior       22
                               Sophomore     5
Tsinghua University            Freshman     17
                               Junior       22
                               Senior       14
                               Sophomore    16
dtype: int64

In [37]:
# 通过get_group方法可以直接获取所在组对应的行，此时必须知道组的具体名字
gb.get_group(('Fudan University', 'Freshman')).iloc[:3, :3]

,School,Grade,Name
15,Fudan University,Freshman,Changqiang Yang
28,Fudan University,Freshman,Gaoqiang Qin
63,Fudan University,Freshman,Gaofeng Zhao


#### 4. 分组的三大操作
**聚合、变换和过滤。**

### 二、聚合函数
#### 1. 内置聚合函数
* 一些直接定义在groupby对象的聚合函数，因为它的速度基本都会经过内部的优化，使用功能时应当优先考虑。根据返回标量值的原则，包括如下函数： `max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod`。



In [39]:
# 返回分组后最小值的索引
gb = df.groupby('Gender')['Height']
gb.idxmin()

Gender
Female    143
Male      199
Name: Height, dtype: int64

In [40]:
# 返回0.95分位数
gb.quantile(0.95)

Gender
Female    166.8
Male      185.9
Name: Height, dtype: float64

In [41]:
# 如果传入的数据来源包含多个列时，将按照列进行迭代计算
gb = df.groupby('Gender')[['Height', 'Weight']]
gb.max()

,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0


#### 2. agg方法
上述的内置函数定义了许多方便的函数，但是仍然有以下不便之处：
* 无法同时使用多个函数

* 无法对特定的列使用特定的聚合函数

* 无法使用自定义的聚合函数

* 无法直接对结果的列名在聚合前进行自定义命名

因此可以使用`agg`函数解决以上问题：

* 使用多个函数

当使用多个聚合函数时，需要用列表的形式把内置聚合函数对应的字符串传入，先前提到的所有字符串都是合法的

In [43]:
# 此时列索引为多级索引，第一层为数据源，第二层为使用的聚合方法，
# 分别逐一对列使用聚合，因此结果为6列
gb.agg(['sum', 'idxmax', 'skew'])

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

* 对特定的列使用特定的聚合函数

对于方法和列的特殊对应，可以通过构造字典传入 agg 中实现，其中字典以列名为键，以聚合字符串或字符串列表为值。

In [44]:
gb.agg({'Height':['mean','max'], 'Weight':'count'})

Height        Weight
             mean    max  count
Gender                         
Female  159.19697  170.2    135
Male    173.62549  193.9     54

* 使用自定义函数

在 agg 中可以使用具体的自定义函数，**需要注意传入函数的参数是之前数据源中的列，逐列进行计算**。

In [45]:
gb.agg(lambda x: x.mean()-x.min())

,Height,Weight
Gender,,
Female,13.79697,13.918519
Male,17.92549,21.759259


由于传入的是序列，因此序列上的方法和属性都是可以在函数中使用的，只需保证返回值是标量即可。

In [46]:
def my_func(s):
  res = 'High'
  if s.mean() <= df[s.name].mean():
    res = 'Low'
  return res

In [47]:
gb.agg(my_func)

,Height,Weight
Gender,,
Female,Low,Low
Male,High,High


* 聚合结果重命名

如果想要对聚合结果的列名进行重命名，只需要将上述函数的位置改写成元组，元组的第一个元素为新的名字，第二个位置为原来的函数，包括聚合字符串和自定义函数

In [48]:
 gb.agg([('range', lambda x: x.max()-x.min()), ('my_sum', 'sum')])

Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0

In [49]:
gb.agg({'Height': [('my_func', my_func), 'sum'],
         'Weight': lambda x:x.max()})

Height            Weight
       my_func      sum <lambda>
Gender                          
Female     Low  21014.0     63.0
Male      High   8854.9     89.0

> 另外需要注意，使用对一个或者多个列使用单个聚合的时候，重命名需要加方括号，否则就不知道是新的名字还是手误输错的内置函数字符串：

In [50]:
gb.agg([('my_sum', 'sum')])

,Height,Weight
,my_sum,my_sum
Gender,,
Female,21014.0,6469.0
Male,8854.9,3929.0


In [51]:
gb.agg({'Height': [('my_func', my_func), 'sum'],
         'Weight': [('range', lambda x:x.max())]})

Height          Weight
       my_func      sum  range
Gender                        
Female     Low  21014.0   63.0
Male      High   8854.9   89.0

### 变换和过滤
#### 1. 变换函数与transform方法
变换函数的返回值为同长度的序列，最常用的内置变换函数是累计函数： cumcount/cumsum/cumprod/cummax/cummin ，它们的使用方式和聚合函数类似，只不过完成的是组内累计操作。

In [52]:
gb.cummax().head()

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0


> 当用自定义变换时需要使用 transform 方法，被调用的自定义函数， 其传入值为数据源的序列 ，与 agg 的传入类型是一致的，其最后的返回结果是行列索引与数据源一致的 DataFrame 。

In [53]:
# 对身高和体重进行分组标准化，即减去组均值后除以组的标准差
gb.transform(lambda x: (x-x.mean())/x.std()).head()


,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631


>  transform 只能返回同长度的序列，但事实上还可以返回一个标量，这会使得结果被广播到其所在的整个组，这种 标量广播 的技巧在特征工程中是非常常见的

In [54]:
# 传入返回标量的函数也是可以的
gb.transform('mean').head()

,Height,Weight
0,159.19697,47.918519
1,173.62549,72.759259
2,173.62549,72.759259
3,159.19697,47.918519
4,173.62549,72.759259


#### 2. 组索引与过滤
* 过滤在分组中是对于组的过滤，而索引是对于行的过滤，在第二章中的返回值，无论是布尔列表还是元素列表或者位置列表，本质上都是对于行的筛选，即如果符合筛选条件的则选入结果表，否则不选入。

* 组过滤作为行过滤的推广，指的是如果对一个组的全体所在行进行统计的结果返回 True 则会被保留， False 则该组会被过滤，最后把所有未被过滤的组其对应的所在行拼接起来作为 DataFrame 返回。

* 在 groupby 对象中，定义了 filter 方法进行组的筛选，其中自定义函数的输入参数为数据源构成的 DataFrame 本身，在之前例子中定义的 groupby 对象中，传入的就是 df[['Height', 'Weight']] ，因此所有表方法和属性都可以在自定义函数中相应地使用，同时只需保证自定义函数的返回为布尔值即可。

In [55]:
# 在原表中通过过滤得到所有容量大于100的组
gb.filter(lambda x: x.shape[0] > 100).head()

,Height,Weight
0,158.9,46.0
3,NaN,41.0
5,158.0,51.0
6,162.5,52.0
7,161.9,50.0


### 四、跨列分组
#### 1. apply的使用
apply 的自定义函数传入参数与 filter 完全一致，只不过后者只允许返回布尔值。


In [56]:
def BMI(x):
  Height = x['Height']/100
  Weight = x['Weight']
  BMI_value = Weight/Height**2
  return BMI_value.mean()

In [57]:
gb.apply(BMI)

Gender
Female    18.860930
Male      24.318654
dtype: float64

除了返回标量之外， apply 方法还可以返回一维 Series 和二维 DataFrame:
* 标量情况：结果得到的是 Series ，索引与 agg 的结果一致：

In [58]:
gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]

gb.apply(lambda x: 0)

Gender  Test_Number
Female  1              0
        2              0
        3              0
Male    1              0
        2              0
        3              0
dtype: int64

In [59]:
gb.apply(lambda x: [0, 0]) # 虽然是列表，但是作为返回值仍然看作标量


Gender  Test_Number
Female  1              [0, 0]
        2              [0, 0]
        3              [0, 0]
Male    1              [0, 0]
        2              [0, 0]
        3              [0, 0]
dtype: object

* Series 情况：得到的是 DataFrame ，行索引与标量情况一致，列索引为 Series 的索引：

In [60]:
gb.apply(lambda x: pd.Series([0,0],index=['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

* Series 情况：得到的是 DataFrame ，行索引与标量情况一致，列索引为 Series 的索引

In [61]:
gb.apply(lambda x: pd.Series([0,0],index=['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

*  DataFrame 情况：得到的是 DataFrame ，行索引最内层在每个组原先 agg 的结果索引上，再加一层返回的 DataFrame 行索引，同时分组结果 DataFrame 的列索引和返回的 DataFrame 列索引一致。

In [62]:
gb.apply(lambda x: pd.DataFrame(np.ones((2,2)),
  index = ['a','b'],
  columns=pd.Index([('w','x'),('y','z')])))

w    y
                        x    z
Gender Test_Number            
Female 1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0
Male   1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0

>  apply 函数的灵活性是以牺牲一定性能为代价换得的，除非需要使用跨列处理的分组处理，否则应当使用其他专门设计的 groupby 对象方法，否则在性能上会存在较大的差距。同时，在使用聚合函数和变换函数时，也应当优先使用内置函数，它们经过了高度的性能优化，一般而言在速度上都会快于用自定义函数来实现。

#### Ex1: 汽车数据集
现有一份汽车数据集，其中 Brand, Disp., HP 分别代表汽车品牌、发动机蓄量、发动机输出。

In [64]:
df = pd.read_csv('data/car.csv')
df.head(3)

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63


In [67]:

df.groupby('Country').filter(lambda x:x.shape[0]>2).groupby(
            'Country')['Price'].agg([
            ('CoV', lambda x: x.std()/x.mean()), 
            'mean', 'count'])

,CoV,mean,count
Country,,,
Japan,0.387429,13938.052632,19
Japan/USA,0.240040,10067.571429,7
Korea,0.243435,7857.333333,3
USA,0.203344,12543.269231,26


In [70]:
df.shape[0]

60

In [71]:
condition = ['Head']*20+['Mid']*20+['Tail']*20
df.groupby(condition)['Price'].mean()




Head     9069.95
Mid     13356.40
Tail    15420.65
Name: Price, dtype: float64

In [72]:
res = df.groupby('Type').agg({'Price': ['max'], 'HP': ['min']})

res.columns = res.columns.map(lambda x:'_'.join(x))
print(res)

         Price_max  HP_min
Type                      
Compact      18900      95
Large        17257     150
Medium       24760     110
Small         9995      63
Sporty       13945      92
Van          15395     106


In [73]:
def normalize(s):
  s_min, s_max = s.min(), s.max()
  res = (s - s_min)/(s_max - s_min)
  return res
df.groupby('Type')['HP'].transform(normalize).head()

0    1.00
1    0.54
2    0.00
3    0.58
4    0.80
Name: HP, dtype: float64

In [74]:
df.groupby('Type')[['HP', 'Disp.']].apply(
  lambda x:np.corrcoef(x['HP'].values, x['Disp.'].values)[0,1])

Type
Compact    0.586087
Large     -0.242765
Medium     0.370491
Small      0.603916
Sporty     0.871426
Van        0.819881
dtype: float64